# Preprocesamiento

In [1]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocesamiento

# Carga de Datos stemming eliminacion de stop words

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Descargar el conjunto de stopwords de nltk si no lo tienes
nltk.download('stopwords')

# Inicializar el Stemmer y las stopwords
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Función de preprocesamiento de texto
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r"[^a-z\s]", "", text)
    # Tokenizar, eliminar stopwords y aplicar stemming
    tokens = [stemmer.stem(word) for word in text.split() if word not in stop_words]
    # Unir los tokens procesados
    return " ".join(tokens)

# Cargar los datos
movies_path = "data/rotten_tomatoes_movies.csv"
reviews_path = "data/rotten_tomatoes_critic_reviews.csv"

movies_df = pd.read_csv(movies_path)
reviews_df = pd.read_csv(reviews_path)

# Preprocesar la columna de críticas
print("Valores nulos en 'review_content':", reviews_df['review_content'].isnull().sum())
reviews_df = reviews_df[reviews_df['review_content'].notnull()]
reviews_df['processed_review'] = reviews_df['review_content'].apply(preprocess_text)

# Crear la matriz TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(reviews_df['processed_review'])

# Vincular críticas con títulos de películas
reviews_with_titles = reviews_df.merge(movies_df[['rotten_tomatoes_link', 'movie_title']], 
                                       on='rotten_tomatoes_link', how='left')

# Mostrar las primeras filas para confirmar
print(reviews_with_titles[['movie_title', 'processed_review']].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Valores nulos en 'review_content': 65806


# Vectorizar

## Matriz TF-IDF

In [ ]:
# Crear la matriz TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(reviews_df['processed_review'])

# Convertir la matriz TF-IDF a un DataFrame para facilitar la visualización
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Mostrar las primeras filas de la matriz TF-IDF
print(tfidf_df.head())

# Motor de busqueda 

## Distancia Jaccard

In [ ]:
# Calcular el índice de Jaccard
def jaccard_similarity(query_tokens, review_tokens):
    # Convertir los tokens en sets
    set_query = set(query_tokens)
    set_review = set(review_tokens)
    
    # Calcular la intersección y la unión de los sets
    intersection = len(set_query.intersection(set_review))
    union = len(set_query.union(set_review))
    
    # Calcular la similitud de Jaccard
    if union == 0:
        return 0
    else:
        return intersection / union

# Función para procesar la consulta del usuario
def process_user_query(query, reviews_df):
    # Preprocesar la consulta del usuario
    query_tokens = preprocess_text(query)
    
    # Crear una lista para almacenar las similitudes
    similarities = []
    
    # Calcular la similitud de Jaccard para cada reseña
    for index, row in reviews_df.iterrows():
        review_tokens = row['processed_review']
        similarity = jaccard_similarity(query_tokens, review_tokens)
        similarities.append((row['movie_title'], similarity))
    
    # Ordenar las reseñas por similitud (de mayor a menor)
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Mostrar las 5 reseñas más similares
    return similarities[:5]

# Cargar los datos
movies_path = "data/rotten_tomatoes_movies.csv"
reviews_path = "data/rotten_tomatoes_critic_reviews.csv"

movies_df = pd.read_csv(movies_path)
reviews_df = pd.read_csv(reviews_path)

# Preprocesar las críticas
reviews_df['processed_review'] = reviews_df['review_content'].apply(preprocess_text)

# Ejemplo de consulta del usuario
user_query = "Amazing movie with great action and plot"

# Obtener las 5 reseñas más similares
top_reviews = process_user_query(user_query, reviews_df)

# Mostrar las reseñas más similares
for title, similarity in top_reviews:
    print(f"Movie: {title}, Similarity: {similarity:.4f}")
